<a href="https://colab.research.google.com/github/Xi123580/Product-Information-Conversion/blob/main/Product_Data_Converter_v2_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
# from time import perf_counter
from google.colab import files
import io
# import time
import datetime
import os


pd.set_option('max_columns', None)

# 1. read file
path = 'https://github.com/Xi123580/Product-Information-Conversion/blob/main/%E7%94%A2%E5%93%81%E8%B3%87%E8%A8%8A_%E5%8E%9F%E5%A7%8B%E8%B3%87%E6%96%99_%E6%AC%84%E4%BD%8D%E8%BD%89%E6%8F%9B%E8%A1%A8_%E8%B3%87%E6%96%99%E8%BD%89%E6%8F%9B%E8%A1%A8_20220421_1.xlsx?raw=true'
df_rawData = pd.read_excel(path, sheet_name = 0)
# print(df_rawData.head().to_string())


In [2]:
# Page: 3~8
def initial_sheet(Page:int):
  df = pd.read_excel(path, sheet_name = Page, usecols = "A, B", dtype=str)
  df.rename(columns={'原始資訊': 'Keys', '網頁呈現字眼': 'Values'}, inplace=True)
  df = df.drop(0)
  df = df.reset_index()
  df = df.drop(columns="index")

  column_SAPSBU = []
  column_Product = []
  ConvertDict = {}
  for i in range(len(df.values[:])):
    column_SAPSBU.append(df.values[i][0])
    column_Product.append(df.values[i][1])
    ConvertDict[column_SAPSBU[i]] = column_Product[i]
  return(ConvertDict)

In [3]:
Rule_SAPSBU = initial_sheet(2)
Rule_IMFRMC = initial_sheet(3)
Rule_IMCMTH = initial_sheet(4)
Rule_IFCI = initial_sheet(5)
Rule_SAPNOR = initial_sheet(6)
Rule_IPURC = initial_sheet(7)

Rule_IMCMTH.update({"07": "已停產"})
# print(Rule_SAPSBU)
# print(Rule_IMFRMC)
# print(Rule_IMCMTH)
# print(Rule_IFCI)
# print(Rule_SAPNOR)
# print(Rule_IPURC)

{'034': 'LSB_034 ELISAs & Specialty Assays', '035': 'DX_035 Detection systems and reagents', '036': 'DX_036 Dekalb CMO', '037': 'DX_037 Oligo Controls and Reference Materials', '052': 'AA_052 Extractions- F&F', '053': 'AA_053 Natural - F&F', '054': 'AA_054 Synthetics - F&F', '056': 'DX_056 Diagnostic Beads', '057': 'DX_057 Magprep', '058': 'DX_058 PROCLIN', '063': 'BM_063 In Process Testing Pharma', '064': 'BM_064 In Process Testing Food&Beverages', '065': 'BM_065 Release Testing Pharma', '066': 'BM_066 Release Testing Food&Beverages', '067': 'BM_067 Environmental Monitoring Pharma', '068': 'BM_068 Environmental Monitoring Food&Beverages', '069': 'BM_069 Media Sigma', '072': 'BM_072 In Process Products Medical', '073': 'BM_073 BioMonitoring Services', '077': 'PS-GENE_077 IP Outlicensing', '090': 'ConLab_090 Digital Lab Applications', '091': 'ConLab_091 BSSN Products', '092': 'ConLab_092 Other Digital Lab Platforms', '093': 'ConLab_093 Connected Lab Growth Initiatives', '480': 'LSC_480 

# 上傳檔案

In [4]:
# upload file
print("請上傳CSV檔 (副檔名須為.csv): ")
uploaded = files.upload()
filename = next(iter(uploaded))
A = uploaded.copy()

請上傳CSV檔 (副檔名須為.csv): 


Saving IIM_Product_List_utf8v2.csv to IIM_Product_List_utf8v2.csv


In [ ]:
A = pd.read_csv(io.BytesIO(uploaded.get(f'{filename}')), dtype=str, encoding = "utf-8-sig", skipinitialspace = True)
# print(A[31650:31750][:].to_string())

In [10]:
# 3. build table
New_df = A.copy()

print(New_df.keys())
print(New_df.shape)
Number_columns = New_df.shape[1]
Number_row = New_df.shape[0]
New_df = New_df.fillna(0)

# figure out table
for col in range(Number_columns):
  Col_Name = df_rawData.keys()[col]
  if Col_Name == "SAPSBU":
    New_df[Col_Name] = New_df[Col_Name].apply(str)
    for row in range(Number_row):
      data = New_df.at[row, Col_Name].strip()
      if data in Rule_SAPSBU:
        x = Rule_SAPSBU[data]
        New_df.at[row, Col_Name] = x
      else:
        New_df.at[row, Col_Name] = "-"
  elif Col_Name == "IMFRMC":
    New_df[Col_Name] = New_df[Col_Name].apply(str)
    for row in range(Number_row):
      data = New_df.at[row, Col_Name].strip()
      if data in Rule_IMFRMC:
        x = Rule_IMFRMC[data]
        New_df.at[row, Col_Name] = x
      elif data == "0" or data.isspace() or len(data) == 0:
        New_df.at[row, Col_Name] = "無管制_但請再確認注意事項"
      else:
        New_df.at[row, Col_Name] = "請洽默克窗口"
  elif Col_Name == "IFCI":
    New_df[Col_Name] = New_df[Col_Name].apply(str)
    for row in range(Number_row):
      data = New_df.at[row, Col_Name].strip()
      if data in Rule_IFCI:
        x = Rule_IFCI[data]
        New_df.at[row, Col_Name] = x
      elif data == "0" or data.isspace() or len(data) == 0:
        New_df.at[row, Col_Name] = "-"
      else:
        New_df.at[row, Col_Name] = "請洽默克窗口"
  elif Col_Name == "IMCMTH":
    New_df[Col_Name] = New_df[Col_Name].apply(str)
    for row in range(Number_row):
      data = New_df.at[row, Col_Name].strip()
      if data in Rule_IMCMTH:
        x = Rule_IMCMTH[data]
        New_df.at[row, Col_Name] = x
      elif data == "0" or data.isspace() or len(data) == 0:
        New_df.at[row, Col_Name] = "-"
      else:
        New_df.at[row, Col_Name] = "請洽默克窗口"
  elif Col_Name == "SAPNOR":
    New_df[Col_Name] = New_df[Col_Name].apply(str)
    for row in range(Number_row):
      data = New_df.at[row, Col_Name].strip()
      if data in Rule_SAPNOR:
        x = Rule_SAPNOR[data]
        New_df.at[row, Col_Name] = x
      else:
        New_df.at[row, Col_Name] = "-"
  elif Col_Name == "IPURC":
    New_df[Col_Name] = New_df[Col_Name].apply(str)
    for row in range(Number_row):
      data = New_df.at[row, Col_Name].strip()
      if data in Rule_IPURC:
        x = Rule_IPURC[data]
        New_df.at[row, Col_Name] = x
      elif data == "0" or data.isspace() or len(data) == 0:
        New_df.at[row, Col_Name] = "Blue"
      else:
        New_df.at[row, Col_Name] = "Blue"    
  else :
    pass

# Change Title
# New_df = New_df.rename(Translation, axis=1)
New_df = New_df[['IPROD', 'IDESC', 'IMNOT1', 'IDRAW', 'SAPSBU', 'ILIST', 'IMFRMC',
       'IFCI', 'IMCMTH', 'SAPNOR', 'IPURC']]

# print(New_df.head().to_string())

Index(['IPROD', 'IDESC', 'IMNOT1', 'IDRAW', 'SAPSBU', 'ILIST', 'IMFRMC',
       'IFCI', 'IMCMTH', 'SAPNOR', 'IPURC'],
      dtype='object')
(279477, 11)


In [ ]:
# print(New_df[270000:][:].to_string())

In [ ]:
# 4. save file as .csv and output.

New_df.to_csv(path_or_buf = "ProductData.csv", sep = ",", na_rep = "",
      index = False, encoding = "utf-8-sig")

Today = datetime.date.today()
os.rename("/content/ProductData.csv", f"ProductData_{Today}.csv")  
files.download(f"ProductData_{Today}.csv")

print("Finish")